In [3]:
import pandas as pd
cc_apps = pd.read_csv("S:\Projects\SD\Credit Card Fraud Detection\cc_approvals.data")
print(cc_apps.head())

   b  30.83      0  u  g  w  v  1.25  t t.1  01  f g.1  00202  0.1  +
0  a  58.67  4.460  u  g  q  h  3.04  t   t   6  f   g  00043  560  +
1  a  24.50  0.500  u  g  q  h  1.50  t   f   0  f   g  00280  824  +
2  b  27.83  1.540  u  g  w  v  3.75  t   t   5  t   g  00100    3  +
3  b  20.17  5.625  u  g  w  v  1.71  t   f   0  f   s  00120    0  +
4  b  32.08  4.000  u  g  m  v  2.50  t   f   0  t   g  00360    0  +


In [4]:
cc_apps_description = cc_apps.describe()
print(cc_apps_description)

print("\n")

# Printing DataFrame information
cc_apps_info = cc_apps.info()
print(cc_apps_info)

print("\n")

# Inspecting the missing values in the dataset
print(cc_apps.tail(17))

                0        1.25          01            0.1
count  689.000000  689.000000  689.000000     689.000000
mean     4.765631    2.224819    2.402032    1018.862119
std      4.978470    3.348739    4.866180    5213.743149
min      0.000000    0.000000    0.000000       0.000000
25%      1.000000    0.165000    0.000000       0.000000
50%      2.750000    1.000000    0.000000       5.000000
75%      7.250000    2.625000    3.000000     396.000000
max     28.000000   28.500000   67.000000  100000.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689 entries, 0 to 688
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   b       689 non-null    object 
 1   30.83   689 non-null    object 
 2   0       689 non-null    float64
 3   u       689 non-null    object 
 4   g       689 non-null    object 
 5   w       689 non-null    object 
 6   v       689 non-null    object 
 7   1.25    689 non-null    float64
 8   t    

In [5]:
import numpy as np

# Inspecting the missing values in the dataset
print(cc_apps.tail(17))

# Replace the '?'s with NaN
cc_apps = cc_apps.replace('?',np.NaN)

# Inspecting the missing values again
print(cc_apps.tail(17))

     b  30.83       0  u  g   w   v   1.25  t t.1  01  f g.1  00202  0.1  +
672  ?  29.50   2.000  y  p   e   h  2.000  f   f   0  f   g  00256   17  -
673  a  37.33   2.500  u  g   i   h  0.210  f   f   0  f   g  00260  246  -
674  a  41.58   1.040  u  g  aa   v  0.665  f   f   0  f   g  00240  237  -
675  a  30.58  10.665  u  g   q   h  0.085  f   t  12  t   g  00129    3  -
676  b  19.42   7.250  u  g   m   v  0.040  f   t   1  f   g  00100    1  -
677  a  17.92  10.210  u  g  ff  ff  0.000  f   f   0  f   g  00000   50  -
678  a  20.08   1.250  u  g   c   v  0.000  f   f   0  f   g  00000    0  -
679  b  19.50   0.290  u  g   k   v  0.290  f   f   0  f   g  00280  364  -
680  b  27.83   1.000  y  p   d   h  3.000  f   f   0  f   g  00176  537  -
681  b  17.08   3.290  u  g   i   v  0.335  f   f   0  t   g  00140    2  -
682  b  36.42   0.750  y  p   d   v  0.585  f   f   0  f   g  00240    3  -
683  b  40.58   3.290  u  g   m   v  3.500  f   f   0  t   s  00400    0  -
684  b  21.0

In [6]:
# assighning the missing values with mean imputation

for i in [2,7,10,14]:
    cc_apps.iloc[:,i].fillna(np.mean(cc_apps.iloc[:,i]),inplace=True)

# Counting the number of NaNs in the dataset to verify
print(cc_apps.iloc[:,[2,7,10,14]].isna().nunique())

0       1
1.25    1
01      1
0.1     1
dtype: int64


In [11]:
columns_to_check = [0, 1, 3, 4, 5, 6, 13]
for col in columns_to_check:
    # Ensure the column index is within the bounds of the dataset
    if col < len(cc_apps.columns):
# Checking if the column is of object type
        if cc_apps.iloc[:, col].dtype == 'object':
            most_frequent_value = cc_apps.iloc[:, col].value_counts().idxmax()
            cc_apps.iloc[:, col] = cc_apps.iloc[:, col].fillna(most_frequent_value)
# Counting the number of NaNs in the dataset and print the counts to verify
print(cc_apps.isna().sum())

b        0
30.83    0
0        0
u        0
g        0
w        0
v        0
1.25     0
t        0
t.1      0
01       0
f        0
g.1      0
00202    0
0.1      0
+        0
dtype: int64


In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cc_apps.columns:
# Comparing if the dtype is object or not
    if cc_apps[col].dtypes=='object':
        cc_apps[col]=le.fit_transform(cc_apps[col])

In [15]:
from sklearn.preprocessing import MinMaxScaler
cc_apps = cc_apps.drop(cc_apps.iloc[:,[11, 13]], axis=1)
cc_apps = cc_apps.values
X,y = cc_apps[:,0:-1] , cc_apps[:,-1]
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(X)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(rescaledX,y, test_size=.33,random_state=42)

In [21]:
from sklearn.linear_model import LogisticRegression
# Applying a LogisticRegression classifier with default parameter values
logreg = LogisticRegression()
logreg.fit(X,y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [18]:
from sklearn.metrics import confusion_matrix
# Useing logreg to predict instances from the test set and store it
y_pred = logreg.predict(X_test)
print("Accuracy of logistic regression classifier: ", ...)
print(confusion_matrix(y_test,y_pred))

Accuracy of logistic regression classifier:  Ellipsis
[[ 90  10]
 [ 26 102]]


In [19]:
from sklearn.model_selection import GridSearchCV
tol = [.01,.001,.0001]
max_iter = [100,150,200]
param_grid = {'tol':tol,'max_iter':max_iter}

In [20]:
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)
# Fit a data to grid_model
grid_model_result = grid_model.fit(rescaledX, y)
# Summarizing the final results
best_score, best_params = grid_model_result.best_score_,grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

Best: 0.850629 using {'max_iter': 100, 'tol': 0.01}
